<a href="https://colab.research.google.com/github/MatinMarjani/ABM-WNV/blob/main/WNV_ABM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet mesa

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import random
import math

# Mosquito and Bird Data Handler
## Here we import and handle the prepared GIS data for mosquitos and bird to include in our model

In [ ]:
import numpy as np
import pandas as pd

class GISDataHandler:
    def __init__(self, bird_data_path, mosquito_data_path):
        # Load the bird and mosquito data
        self.bird_data = self.load_data(bird_data_path)
        self.mosquito_data = self.load_data(mosquito_data_path)

    def load_data(self, data_path):
        # Assuming data is in CSV format, each file containing 52 matrices (one per week)
        # with each row representing a flattened grid for a week.
        data = pd.read_csv(data_path, header=None)
        weeks = []
        for week in range(52):
            week_data = data.iloc[week].values.reshape(-1, int(np.sqrt(data.shape[1])))
            weeks.append(week_data)
        return np.array(weeks)

    def get_abundance(self, x, y, week, agent_type):
        if agent_type == "bird":
            return self.bird_data[week, x, y]
        elif agent_type == "mosquito":
            return self.mosquito_data[week, x, y]
        else:
            raise ValueError("Invalid agent type. Use 'bird' or 'mosquito'.")

# Example usage:
# gis_handler = GISDataHandler('bird_data.csv', 'mosquito_data.csv')
# bird_abundance = gis_handler.get_abundance(x, y, week, 'bird')
# mosquito_abundance = gis_handler.get_abundance(x, y, week, 'mosquito')


# Helper functions

## We add the parameters used in the model here.

In [ ]:
# Helper functions for temperature-dependent parameters
def k_T(T):
    return 0.344 / (1 + 1.231e-0.1347*(T-20))

def bL_T(T):
    return 2.325 * k_T(T)

def bM_T(T):
    return 0.1 * bL_T(T)

def mL_T(T):
    return 0.0025 * T**2 - 0.094 * T + 1.0257

def mM_T(T):
    return 0.1 * mL_T(T)

def betaM_T(T):
    return 0.75 * k_T(T)

def gammaM_T(T):
    return 1 / (0.00937 * T - 0.1352) if T > 15 else 0

# Mosquito Agent


In [ ]:

Copy code
class Mosquito(Agent):
    def __init__(self, unique_id, model, state="larvae"):
        super().__init__(unique_id, model)
        self.state = state

    def step(self):
        T = self.model.temperature
        x, y = self.pos
        week = self.model.schedule.steps % 52  # Determine current week in the simulation
        abundance = self.model.gis_data.get_abundance(x, y, week, agent_type="mosquito")

        if self.state == "larvae":
            if random.random() < bM_T(T) and abundance > 0:
                self.state = "susceptible"
            elif random.random() < mL_T(T):
                self.state = "dead"
        elif self.state == "susceptible":
            self.move()
            if random.random() < mM_T(T):
                self.state = "dead"
            elif self.model.check_infection(self):
                self.state = "exposed"
        elif self.state == "exposed":
            if random.random() < gammaM_T(T):
                self.state = "infectious"
            elif random.random() < mM_T(T):
                self.state = "dead"
        elif self.state == "infectious":
            self.move()
            if random.random() < mM_T(T):
                self.state = "dead"

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

# Bird Agent

In [ ]:
class Bird(Agent):
    def __init__(self, unique_id, model, state="susceptible"):
        super().__init__(unique_id, model)
        self.state = state

    def step(self):
        T = self.model.temperature
        x, y = self.pos
        week = self.model.schedule.steps % 52  # Determine current week in the simulation
        abundance = self.model.gis_data.get_abundance(x, y, week, agent_type="bird")

        if self.state == "susceptible":
            self.move()
            if self.model.check_infection(self):
                self.state = "exposed"
        elif self.state == "exposed":
            if random.random() < self.model.gammaB:
                self.state = "infectious"
            elif random.random() < self.model.mB:
                self.state = "dead"
        elif self.state == "infectious":
            if random.random() < self.model.alphaB:
                self.state = "dead"
            elif random.random() < (1 - self.model.nuB):
                self.state = "recovered"
        elif self.state == "recovered":
            if random.random() < self.model.mB:
                self.state = "dead"

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

# Human Agent

In [ ]:
class Human(Agent):
    def __init__(self, unique_id, model, state="susceptible"):
        super().__init__(unique_id, model)
        self.state = state

    def step(self):
        self.move()
        # Infection dynamics for humans can be added here

    '''def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)'''

# Main Model

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import random
import numpy as np

# Updated model class
class WNVModel(Model):
    def __init__(self, width, height, N, temperature, bird_data_path, mosquito_data_path):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.temperature = temperature
        self.gis_data = GISDataHandler(bird_data_path, mosquito_data_path)

        # Constants
        self.gammaB = 0.333  # Rate infected-infectious birds
        self.mB = 0.001404  # Mortality rate of birds
        self.alphaB = 0.28  # Removal rate of birds due to infection
        self.nuB = 0.43  # Portion of birds dying due to infection

        # Create agents
        for i in range(self.num_agents):
            agent_type = random.choice(["mosquito", "bird", "human"])
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            if agent_type == "mosquito":
                abundance = self.gis_data.get_abundance(x, y, week=0, agent_type="mosquito")
                state = "larvae" if abundance > 0 else "dead"
                agent = Mosquito(i, self, state)
            elif agent_type == "bird":
                abundance = self.gis_data.get_abundance(x, y, week=0, agent_type="bird")
                state = "susceptible" if abundance > 0 else "dead"
                agent = Bird(i, self, state)
            else:
                agent = Human(i, self)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (x, y))

        # Create a CSV file to save results
        self.results_file = open('simulation_results.csv', 'w', newline='')
        self.csv_writer = csv.writer(self.results_file)
        self.csv_writer.writerow(['step', 'agent_id', 'agent_type', 'x', 'y', 'state'])

    def step(self):
        self.schedule.step()
        self.save_results()

    def check_infection(self, agent):
        neighbors = self.grid.get_neighbors(agent.pos, moore=True, include_center=False)
        if isinstance(agent, Mosquito):
            for neighbor in neighbors:
                if isinstance(neighbor, Bird) and neighbor.state == "infectious":
                    return random.random() < betaM_T(self.temperature)
        elif isinstance(agent, Bird):
            for neighbor in neighbors:
                if isinstance(neighbor, Mosquito) and neighbor.state == "infectious":
                    return random.random() < betaM_T(self.temperature)
        return False


     def save_results(self):
        for agent in self.schedule.agents:
            self.csv_writer.writerow([self.schedule.steps, agent.unique_id, type(agent).__name__, agent.pos[0], agent.pos[1], agent.state])

     def __del__(self):
        self.results_file.close()


# Simulation

In [ ]:
# Running the model
if __name__ == "__main__":
    model = WNVModel(10, 10, 50, 25)

    # Run the model for 10 steps
    for i in range(10):
        model.step()

# Results and visualization of the run

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def animate_simulation(model, num_steps):
    fig, ax = plt.subplots()
    ims = []

    for step in range(num_steps):
        model.step()
        agents = model.schedule.agents
        x_mosquitoes = [agent.pos[0] for agent in agents if isinstance(agent, Mosquito)]
        y_mosquitoes = [agent.pos[1] for agent in agents if isinstance(agent, Mosquito)]
        x_birds = [agent.pos[0] for agent in agents if isinstance(agent, Bird)]
        y_birds = [agent.pos[1] for agent in agents if isinstance(agent, Bird)]
        x_humans = [agent.pos[0] for agent in agents if isinstance(agent, Human)]
        y_humans = [agent.pos[1] for agent in agents if isinstance(agent, Human)]

        im = ax.scatter(x_mosquitoes, y_mosquitoes, c='r', label='Mosquitoes', alpha=0.6)
        im = ax.scatter(x_birds, y_birds, c='g', label='Birds', alpha=0.6)
        im = ax.scatter(x_humans, y_humans, c='b', label='Humans', alpha=0.6)
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=200, blit=True, repeat_delay=1000)
    plt.legend(loc='upper right')
    plt.title('Agent-based Model Simulation')
    plt.show()

# Running the model with visualization
if __name__ == "__main__":
    model = WNVModel(10, 10, 50, 25, 'bird_data.csv', 'mosquito_data.csv')
    animate_simulation(model, num_steps=10)
